In [1]:
import pandas as pd
import numpy as np
import pymongo
import warnings
import time
import random
from tqdm.notebook import tqdm
import itertools
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score, recall_score, precision_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# CONEXIÓN

In [2]:
CONNECTION_STRING = "mongodb://proyectoestudiantes:suVKLoKoqjCxdeRnmZH7IICunDq54Ed33zaKcNzPBdxI2PVaohC0veT5diWHmsojaQCW6r2qohC9ACDbu5vPqQ==@proyectoestudiantes.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@proyectoestudiantes@"

In [ ]:
client = pymongo.MongoClient(CONNECTION_STRING)
db = client['Estudiantes']
collection = db['Estudiantes_Materias']

datos_raw = list(collection.find({}))

registros = []
for doc in datos_raw:
    datos_personales = doc.get('datos_personales', {})
    info_academica = doc.get('academico', {})
    location = doc.get('location', {})
    info_colegio = doc.get('colegio', {})
    puntajes = doc.get('ICFES', {})
    metricas = doc.get('metricas_rendimiento', {})
    estado = doc.get('estado', {})
    periodo_info = doc.get('periodo_info', {})
    
    registro = {
        'edad': datos_personales.get('edad'),
        'genero': datos_personales.get('genero'),
        'estrato': datos_personales.get('estrato'),
        'discapacidad': datos_personales.get('discapacidad'),
        'programa': info_academica.get('programa'),
        'programa_secundario': info_academica.get('programa_secundario'),
        'tiene_programa_secundario': 1 if info_academica.get('programa_secundario') else 0,
        'semestre_actual': info_academica.get('semestre_actual'),
        'tipo_estudiante': info_academica.get('tipo_estudiante'),
        'tipo_admision': info_academica.get('tipo_admision'),
        'estado_academico': info_academica.get('estado_academico'),
        'ciudad_residencia': location.get('ciudad'),
        'depto_residencia': location.get('departamento'),
        'pais': location.get('pais'),
        'es_barranquilla': location.get('es_barranquilla'),
        'es_colombia': location.get('es_colombia'),
        'codigo_dane': location.get('codigo_dane'),
        'tipo_colegio': info_colegio.get('tipo_colegio'),
        'calendario_colegio': info_colegio.get('calendario_colegio'),
        'descripcion_bachillerato': info_colegio.get('descripcion_bachillerato'),
        'puntaje_total': puntajes.get('puntaje_total'),
        'matematicas': puntajes.get('matematicas'),
        'lectura_critica': puntajes.get('lectura_critica'),
        'sociales': puntajes.get('sociales'),
        'ciencias': puntajes.get('ciencias'),
        'ingles': puntajes.get('ingles'),
        'promedio': metricas.get('promedio_acumulado'),
        'materias_cursadas': metricas.get('materias_cursadas_total'),
        'materias_perdidas': metricas.get('materias_perdidas_total'),
        'materias_repetidas': metricas.get('materias_repetidas'),
        'perdidas_por_depto': metricas.get('materias_perdidas_por_departamento', {}),
        'beca': estado.get('becado'),
        'graduado': estado.get('graduado'),
        'desertor': estado.get('desertor'),
        'ultimo_periodo': periodo_info.get('ultimo_periodo')
    }
    registros.append(registro)

df = pd.DataFrame(registros)
df = df[df['graduado'] == False]

print(f"Datos extraidos: {df.shape[0]} estudiantes x {df.shape[1]} variables")
print(f"Desertores: {df['desertor'].sum()} ({df['desertor'].sum()/len(df)*100:.2f}%)")

client.close()

Datos extraidos: 9455 estudiantes x 35 variables
Desertores: 477 (5.04%)


In [22]:
df

,edad,genero,estrato,discapacidad,programa,programa_secundario,tiene_programa_secundario,semestre_actual,tipo_estudiante,tipo_admision,...,ingles,promedio,materias_cursadas,materias_perdidas,materias_repetidas,perdidas_por_depto,beca,graduado,desertor,ultimo_periodo
0,17,F,5,00- Ninguno,Derecho,None,0,1.0,Estudiante regular,Ordinaria Pregrado,...,54.0,3.47,13,9,1,"{'Dpto. Derecho': 7, 'Dpto. de Economía': 1, '...",No becado,0,0,202510
1,18,M,1,00- Ninguno,Geología,None,0,1.0,Estudiante regular,Ordinaria Pregrado,...,56.0,3.82,11,5,1,"{'Dpto. Español': 1, 'Dpto. Historia y Cs. Soc...",No becado,0,0,202510
2,17,F,2,00- Ninguno,Odontología,None,0,1.0,Estudiante regular,Ordinaria Pregrado,...,100.0,4.03,9,6,0,"{'Dpto. Química y Biología': 2, 'Dpto. Español...",Institucional,0,0,202510
3,18,M,4,00- Ninguno,Ingeniería Industrial,None,0,1.0,Estudiante regular,Ordinaria Pregrado,...,78.0,4.18,10,5,0,"{'Dpto. Español': 1, 'Dpto. Ing. Civil y Ambie...",Institucional,0,0,202510
4,18,M,4,00- Ninguno,Economía,None,0,1.0,Estudiante regular,Ordinaria Pregrado,...,72.0,3.52,11,5,0,"{'Dpto. Español': 1, 'Dpto. de Economía': 2, '...",No becado,0,0,202510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219,24,M,5,00- Ninguno,Diseño Gráfico,None,0,5.0,Estudiante regular,Ordinaria Pregrado,...,NaN,4.56,16,5,0,"{'Dpto. Historia y Cs. Sociales': 1, 'Dpto. Di...",No becado,0,0,202510
10221,24,F,4,00- Ninguno,Ingeniería Mecánica,None,0,7.0,Estudiante regular,Transferencia Externa,...,NaN,4.68,16,5,0,"{'Dpto.Ing Eléctrica-Electrónica': 1, 'Dpto. L...",No becado,0,0,202510
10222,22,M,4,00- Ninguno,Psicología,None,0,5.0,Estudiante regular,Ordinaria Pregrado,...,NaN,4.14,20,7,0,"{'Dpto. Humanidades y Filosofía': 1, 'Dpto. Le...",No becado,0,0,202510
10223,24,F,4,00- Ninguno,Medicina,None,0,11.0,Estudiante regular,Ordinaria Pregrado,...,NaN,4.18,9,4,0,"{'Dpto. Salud Pública': 1, 'Dpto. Medicina': 3}",No becado,0,0,202510


# Preprocesamiento

In [23]:
perdidas_df = pd.json_normalize(df['perdidas_por_depto'])
perdidas_df = perdidas_df.add_prefix('perdidas_')
df = pd.concat([df.drop('perdidas_por_depto', axis=1), perdidas_df], axis=1)

cols_perdidas_depto = [col for col in df.columns if col.startswith('perdidas_')]

df_desertores = df[df['desertor'] == 1].copy()
df_no_desertores = df[df['desertor'] == 0].copy()

numericas = df.select_dtypes(include=[np.number]).columns.tolist()
numericas = [col for col in numericas if col not in ['desertor', 'graduado']]

for col in numericas:
    if df[col].isna().sum() > 0:
        if col in cols_perdidas_depto or col == 'semestre_actual':
            # Llenar con 0: perdidas por departamento y semestre actual vacio
            df[col] = df[col].fillna(0)
        else:
            mediana_desertores = df_desertores[col].median()
            mediana_no_desertores = df_no_desertores[col].median()
            df.loc[(df['desertor'] == 1) & (df[col].isna()), col] = mediana_desertores
            df.loc[(df['desertor'] == 0) & (df[col].isna()), col] = mediana_no_desertores

categoricas = ['genero', 'discapacidad', 'programa', 'programa_secundario', 
               'tipo_estudiante', 'tipo_admision', 'estado_academico',
               'ciudad_residencia', 'depto_residencia', 'pais', 
               'tipo_colegio', 'calendario_colegio', 'descripcion_bachillerato',
               'ultimo_periodo']

for col in categoricas:
    if col in df.columns and df[col].isna().sum() > 0:
        moda_desertores = df[df['desertor'] == 1][col].mode()
        moda_no_desertores = df[df['desertor'] == 0][col].mode()
        valor_desertores = moda_desertores[0] if len(moda_desertores) > 0 else 'Desconocido'
        valor_no_desertores = moda_no_desertores[0] if len(moda_no_desertores) > 0 else 'Desconocido'
        df.loc[(df['desertor'] == 1) & (df[col].isna()), col] = valor_desertores
        df.loc[(df['desertor'] == 0) & (df[col].isna()), col] = valor_no_desertores

encoders = {}
for col in categoricas:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        encoders[col] = le

cols_object = df.select_dtypes(include=['object']).columns.tolist()
cols_object = [col for col in cols_object if col not in ['desertor', 'graduado']]

if cols_object:
    for col in cols_object:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        encoders[col] = le

print(f"Datos procesados: {df.shape[0]} filas x {df.shape[1]} columnas")

Datos procesados: 10215 filas x 62 columnas


# partición

In [24]:
df = df.dropna(subset=['desertor'])

X = df.drop(['desertor', 'graduado'], axis=1)
y = df['desertor'].astype(int)

# Primera división: 70% entrenamiento, 30% temporal
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Segunda división: dividir el 30% temporal en 15% validación y 15% prueba
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Escalar datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {X_train_scaled.shape[0]} ({X_train_scaled.shape[0]/len(df)*100:.1f}%)")
print(f"Validacion: {X_val_scaled.shape[0]} ({X_val_scaled.shape[0]/len(df)*100:.1f}%)")
print(f"Test: {X_test_scaled.shape[0]} ({X_test_scaled.shape[0]/len(df)*100:.1f}%)")
print(f"Features: {X_train_scaled.shape[1]}")
print(f"\nDistribucion desertores:")
print(f"Train: {y_train.sum()} ({y_train.sum()/len(y_train)*100:.1f}%)")
print(f"Validacion: {y_val.sum()} ({y_val.sum()/len(y_val)*100:.1f}%)")
print(f"Test: {y_test.sum()} ({y_test.sum()/len(y_test)*100:.1f}%)")

Train: 6618 (70.0%)
Validacion: 1418 (15.0%)
Test: 1419 (15.0%)
Features: 59

Distribucion desertores:
Train: 334 (5.0%)
Validacion: 71 (5.0%)
Test: 72 (5.1%)


# Configuración grid search 

In [25]:
# Diccionarios de configuracion para grid search EXPANDIDO
tecnicas_muestreo = {
    'smote_30': SMOTE(sampling_strategy=0.30, random_state=42),
    'smote_40': SMOTE(sampling_strategy=0.40, random_state=42),
    'smote_50': SMOTE(sampling_strategy=0.50, random_state=42),
    'smote_60': SMOTE(sampling_strategy=0.60, random_state=42)
}

class_weights_base = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
cw_base_dict = {0: class_weights_base[0], 1: class_weights_base[1]}

configuraciones_pesos = {
    'sin_pesos': None,
    'balanced': cw_base_dict,
    'high_recall_1.5x': {0: cw_base_dict[0], 1: cw_base_dict[1] * 1.5},
    'high_recall_2x': {0: cw_base_dict[0], 1: cw_base_dict[1] * 2},
    'high_recall_3x': {0: cw_base_dict[0], 1: cw_base_dict[1] * 3}
}

arquitecturas = {
   
    'light_3': [128, 64, 32],
    'medium_4': [512, 256, 128, 64],
    'deep_3': [512, 256, 256, 128, 64]
    
}

hiperparametros = {
    'dropout_rates': [0.2, 0.3, 0.4, 0.5],
    'learning_rates': [ 0.001, 0.002],
    'batch_sizes': [64, 128, 256],
    'optimizers': ['adam', 'rmsprop']
}

regularizaciones = {
    'sin_regularizacion': None,
    'l2_very_light': l2(0.0001),
    'l2_light': l2(0.001)
    
}

thresholds = [0.3, 0.35, 0.4, 0.45, 0.5]



In [39]:
# Generar todas las combinaciones usando los diccionarios
random.seed(42)
np.random.seed(42)

configuraciones_entrenamiento = []

# Recorrer cada diccionario para generar combinaciones
for muestreo_nombre, muestreo_obj in tecnicas_muestreo.items():
    for peso_nombre, peso_obj in configuraciones_pesos.items():
        for arq_nombre, capas in arquitecturas.items():
            for dropout in hiperparametros['dropout_rates']:
                for lr in hiperparametros['learning_rates']:
                    for bs in hiperparametros['batch_sizes']:
                        for opt in hiperparametros['optimizers']:
                            for reg_nombre, reg_obj in regularizaciones.items():
                                config = {
                                    'muestreo_nombre': muestreo_nombre,
                                    'muestreo': muestreo_obj,
                                    'peso_nombre': peso_nombre,
                                    'pesos': peso_obj,
                                    'arquitectura_nombre': arq_nombre,
                                    'capas': capas,
                                    'dropout': dropout,
                                    'learning_rate': lr,
                                    'batch_size': bs,
                                    'optimizer': opt,
                                    'regularizacion_nombre': reg_nombre,
                                    'regularizacion': reg_obj,
                                }
                                configuraciones_entrenamiento.append(config)


# Muestreo aleatorio de configuraciones para hacer el proceso manejable
# Ajusta este número según tu capacidad de cómputo
MAX_CONFIGS = 100



print(f"Configuraciones generadas: {len(configuraciones_entrenamiento)}")
if len(configuraciones_entrenamiento) > MAX_CONFIGS:
    random.shuffle(configuraciones_entrenamiento)
    configuraciones_entrenamiento = configuraciones_entrenamiento[:MAX_CONFIGS]
print("solo  se entrenaran 100 modelos aleatorios de las configuraciones posibles")

Configuraciones generadas: 8640
solo  se entrenaran 100 modelos aleatorios de las configuraciones posibles


In [41]:
# Entrenar todos los modelos
resultados_busqueda = []
modelos_entrenados = {}

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=0)



# Barra de progreso
pbar = tqdm(enumerate(configuraciones_entrenamiento, 1), total=len(configuraciones_entrenamiento), 
            desc="Entrenando modelos", unit="modelo")

for idx, config in pbar:
    try:
        # Aplicar tecnica de muestreo
        if config['muestreo'] is not None:
            X_train_prep, y_train_prep = config['muestreo'].fit_resample(X_train_scaled, y_train)
        else:
            X_train_prep = X_train_scaled
            y_train_prep = y_train
        
        # Construir modelo
        modelo = Sequential()
        modelo.add(Dense(config['capas'][0], activation='relu', input_dim=X_train_prep.shape[1], 
                        kernel_regularizer=config['regularizacion']))
        modelo.add(Dropout(config['dropout']))
        
        for units in config['capas'][1:]:
            modelo.add(Dense(units, activation='relu', kernel_regularizer=config['regularizacion']))
            modelo.add(Dropout(config['dropout']))
        
        modelo.add(Dense(1, activation='sigmoid'))
        
        # Configurar optimizador
        if config['optimizer'] == 'adam':
            opt = Adam(learning_rate=config['learning_rate'])
        else:
            opt = RMSprop(learning_rate=config['learning_rate'])
        
        modelo.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        
        # Entrenar
        inicio = time.time()
        history = modelo.fit(
            X_train_prep, y_train_prep, epochs=100, batch_size=config['batch_size'],
            validation_data=(X_val_scaled, y_val), class_weight=config['pesos'], 
            callbacks=[early_stop, reduce_lr], verbose=0
        )
        
        
        # Predecir en validación
        y_pred_proba = modelo.predict(X_val_scaled, verbose=0).flatten()
        
        # Guardar modelo
        modelo_id = f"modelo_{idx}"
        modelos_entrenados[modelo_id] = {
            'modelo': modelo,
            'config': config,
            'y_pred_proba': y_pred_proba,
            'history': history
        }
        
        # Evaluar con diferentes thresholds
        for thresh in thresholds:
            y_pred = (y_pred_proba >= thresh).astype(int)
            
            recall = recall_score(y_val, y_pred)
            precision = precision_score(y_val, y_pred, zero_division=0)
            f1 = f1_score(y_val, y_pred)
            auc = roc_auc_score(y_val, y_pred_proba)
            
            cm = confusion_matrix(y_val, y_pred)
            tn, fp, fn, tp = cm.ravel()
            
            score_custom = 0.6 * recall + 0.3 * auc + 0.1 * precision
            
            resultado = {
                'modelo_id': modelo_id, 'config_idx': idx,
                'muestreo': config['muestreo_nombre'], 'pesos': config['peso_nombre'],
                'arquitectura': config['arquitectura_nombre'], 'capas': str(config['capas']),
                'dropout': config['dropout'], 'learning_rate': config['learning_rate'],
                'batch_size': config['batch_size'], 'optimizer': config['optimizer'],
                'regularizacion': config['regularizacion_nombre'], 'threshold': thresh,
                'recall': recall, 'precision': precision, 'f1': f1, 'auc': auc,
                'score_custom': score_custom, 'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn,
                'epochs': len(history.history['loss'])
            }
            resultados_busqueda.append(resultado)
        
        # Actualizar barra de progreso
       
        
        
        
    except Exception as e:
        pbar.write(f"Error en modelo {idx}: {str(e)}")
        continue




Entrenando modelos:   0%|          | 0/100 [00:00<?, ?modelo/s]

KeyboardInterrupt: 